In [1]:
import sys
import os

# Get the project root directory (parent of the notebooks folder)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the src folder to the Python path
sys.path.append(project_root)

In [2]:
from src.data_preprocessing import DataLoaders

loaders = DataLoaders(trial_data=True)
train_dataloader = loaders.get_train_dataloader()
test_dataloader = loaders.get_test_dataloader()
for images, labels in train_dataloader:
    print(f'Batch of images shape: {images.shape}')
    print(f'Batch of labels: {labels}')

for images, labels in test_dataloader:
    print(f'Batch of images shape: {images.shape}')
    print(f'Batch of labels: {labels}')

Batch of images shape: torch.Size([32, 3, 224, 224])
Batch of labels: tensor([ 98, 103, 130, 103,  87,  15,  69,  71, 127, 129,  52, 130,  96,  38,
         57,   2,  75, 121, 130, 130, 130,  38,  54, 116, 130,  87,  90,  75,
        114, 103, 103,  79])
Batch of images shape: torch.Size([13, 3, 224, 224])
Batch of labels: tensor([ 87, 130, 129, 103, 114,  15,  20, 130,  38,  58, 103,  43,  38])
Batch of images shape: torch.Size([5, 3, 224, 224])
Batch of labels: tensor([126, 130, 114,   2,  58])


In [3]:
import torch


def evaluate(model, test_loader, criterion, device):
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

In [4]:

from src.models.tiny_vit import tiny_vit_5m_224
import torch.optim as optim
import torch.nn as nn

num_epochs = 10
learning_rate = 0.001

model = tiny_vit_5m_224()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
device = torch.device("mps")

model.to(device)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_loss = running_loss / len(train_dataloader)
    test_loss, test_accuracy = evaluate(model, test_dataloader, criterion, device)
    print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {train_loss:.4f}, '
              f'Test Loss: {test_loss:.4f}, '
              f'Test Accuracy: {test_accuracy:.2f}%')

/Users/mateuszwitka-jezewski/Documents/Projekty/geoguessrai/src/models/tiny_vit.py:640: UserWarning: Overwriting tiny_vit_5m_224 in registry with src.models.tiny_vit.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
/Users/mateuszwitka-jezewski/Documents/Projekty/geoguessrai/src/models/tiny_vit.py:653: UserWarning: Overwriting tiny_vit_11m_224 in registry with src.models.tiny_vit.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
/Users/mateuszwitka-jezewski/Documents/Projekty/geoguessrai/src/models/tiny_vit.py:666: UserWarning: Overwriting tiny_vit_21m_224 in registry with src.models.tiny_vit.tiny_vit_21m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
/Users/mateuszwitka-jezewski/Documents/Projekty/geogue

Epoch [1/10], Train Loss: 6.8911, Test Loss: 6.3681, Test Accuracy: 0.00%
Epoch [2/10], Train Loss: 4.1856, Test Loss: 5.4524, Test Accuracy: 20.00%
Epoch [3/10], Train Loss: 2.7572, Test Loss: 5.7069, Test Accuracy: 20.00%
Epoch [4/10], Train Loss: 1.7090, Test Loss: 5.0508, Test Accuracy: 20.00%
Epoch [5/10], Train Loss: 1.2259, Test Loss: 6.1780, Test Accuracy: 0.00%
Epoch [6/10], Train Loss: 0.6554, Test Loss: 5.6981, Test Accuracy: 0.00%
Epoch [7/10], Train Loss: 0.4727, Test Loss: 5.1840, Test Accuracy: 0.00%
Epoch [8/10], Train Loss: 0.2835, Test Loss: 5.3708, Test Accuracy: 0.00%
Epoch [9/10], Train Loss: 0.1827, Test Loss: 5.5460, Test Accuracy: 0.00%
Epoch [10/10], Train Loss: 0.1415, Test Loss: 5.5969, Test Accuracy: 0.00%
